In [1]:
import os
import random
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
import torch.optim as opt
import torch.nn.functional as F
from torchvision.io import read_image
from torchvision import transforms
from tqdm.notebook import tqdm
from PIL import Image
import optuna
from optuna.trial import TrialState

In [2]:
random.seed(2020)

In [3]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
device

device(type='cuda', index=0)

In [4]:
resize_image = transforms.Resize(size=(227, 227))

In [5]:
def calculate_mean(dataset):
    mean = torch.Tensor([0.,0.,0.])
    
    num_samples = len(dataset)
    for i in range(num_samples):
        image, _, _, _ = dataset[i]
        image = image.type(torch.FloatTensor) / 255
        
        for j in range(3):
            mean[j] += torch.mean(image[j, :, :])
            
    mean = mean / num_samples
    return mean

In [6]:
def subtract_mean(image, mean):
    normed_image = torch.zeros(image.size())
    image = image.type(torch.FloatTensor) / 255
    
    for j in range(3):
        normed_image[j] = image[j, :, :] - mean[j]
    return normed_image

In [7]:
class AdienceDataset(Dataset):
    def __init__(self, annot_file, img_dir, train=False, transform=None):
        self.img_lbls = pd.read_csv(annot_file, header=None)
        self.img_lbls[2] = [x.split('.')[1] for x in self.img_lbls[0]]
        self.img_dir = img_dir
        self.transform = transform
        self.is_train = train
        self.mean = torch.Tensor([0.,0.,0.])
    
    def __len__(self):
        return len(self.img_lbls)
    
    def __getitem__(self, idx):
        img_file = self.img_lbls.iloc[idx, 0]
        face_id = self.img_lbls.iloc[idx, 2]
        img_path = os.path.join(self.img_dir, img_file)
        image = read_image(img_path)
#         image = resize_image(image)
        label = self.img_lbls.iloc[idx, 1]
    
        if self.is_train:
            try:
                positive_list = self.img_lbls[self.img_lbls.iloc[:, 2] == face_id].index.values
                positive_list = np.setdiff1d(positive_list, np.array([idx]))
                positive_item = random.choice(positive_list)
            except IndexError:
                positive_list = self.img_lbls[self.img_lbls.iloc[:, 1] == label].index.values
                positive_item = random.choice(positive_list)
            positive_img = self.img_lbls.iloc[positive_item, 0]
            pos_img_path = os.path.join(self.img_dir, positive_img)
            pos_image = read_image(pos_img_path)
            
            negative_list = self.img_lbls[self.img_lbls.iloc[:, 1] != label].index.values
            negative_item = random.choice(negative_list)
            negative_img = self.img_lbls.iloc[negative_item, 0]
            neg_img_path = os.path.join(self.img_dir, negative_img)
            neg_image = read_image(neg_img_path)

            if self.transform:
                image = self.transform(image, self.mean)
                pos_image = self.transform(pos_image, self.mean)
                neg_image = self.transform(neg_image, self.mean)
                
            return image, pos_image, neg_image, label
        else:
            if self.transform:
                image = self.transform(image, self.mean)

            return image, label
    
    def set_mean(self, mean):
        self.mean = mean

In [8]:
class AdienceDataset(Dataset):
    def __init__(self, annot_file, img_dir, train=False, transform=None):
        self.img_lbls = pd.read_csv(annot_file, header=None)
#         self.img_lbls[2] = [x.split('.')[1] for x in self.img_lbls[0]]
        self.img_dir = img_dir
        self.transform = transform
        self.is_train = train
        self.mean = torch.Tensor([0.,0.,0.])
    
    def __len__(self):
        return len(self.img_lbls)
    
    def __getitem__(self, idx):
        img_file = self.img_lbls.iloc[idx, 0]
#         face_id = self.img_lbls.iloc[idx, 2]
        img_path = os.path.join(self.img_dir, img_file)
        image = read_image(img_path)
#         image = resize_image(image)
        label = self.img_lbls.iloc[idx, 1]
    
        if self.is_train:
            positive_list = self.img_lbls[self.img_lbls.iloc[:, 1] == label].index.values
            positive_list = np.setdiff1d(positive_list, np.array([idx]))
            positive_item = random.choice(positive_list)
            positive_img = self.img_lbls.iloc[positive_item, 0]
            pos_img_path = os.path.join(self.img_dir, positive_img)
            pos_image = read_image(pos_img_path)
            
            negative_list = self.img_lbls[self.img_lbls.iloc[:, 1] != label].index.values
            negative_item = random.choice(negative_list)
            negative_img = self.img_lbls.iloc[negative_item, 0]
            neg_img_path = os.path.join(self.img_dir, negative_img)
            neg_image = read_image(neg_img_path)

            if self.transform:
                image = self.transform(image, self.mean)
                pos_image = self.transform(pos_image, self.mean)
                neg_image = self.transform(neg_image, self.mean)
                
            return image, pos_image, neg_image, label
        else:
            if self.transform:
                image = self.transform(image, self.mean)

            return image, label
    
    def set_mean(self, mean):
        self.mean = mean

In [14]:
train_data = AdienceDataset("../train.csv", "../cropped_Adience/", train=True)

In [9]:
train_data = AdienceDataset("../train.csv", "../cropped_Adience/", train=True, transform=subtract_mean)
test_data = AdienceDataset("../test.csv", "../cropped_Adience/", train=False, transform=subtract_mean)
val_data = AdienceDataset("../val.csv", "../cropped_Adience/", train=True, transform=subtract_mean)

In [10]:
# mean = calculate_mean(train_data)
mean = torch.Tensor([0.5062, 0.3581, 0.3099])
mean

tensor([0.5062, 0.3581, 0.3099])

In [11]:
train_data.set_mean(mean)
test_data.set_mean(mean)
val_data.set_mean(mean)

In [12]:
class TripletLoss(nn.Module):
    def __init__(self, margin=1.0):
        super(TripletLoss, self).__init__()
        self.margin = margin
        
    def calc_euclidean(self, x1, x2):
        return (x1 - x2).pow(2).sum(1)
    
    def forward(self, anchor, positive, negative):
        distance_positive = self.calc_euclidean(anchor, positive)
        distance_negative = self.calc_euclidean(anchor, negative)
        losses = torch.relu(distance_positive - distance_negative + self.margin)

        return losses.mean()

In [13]:
class CombinedLoss(nn.Module):
    def __init__(self, alpha=1.0, beta=1.0):
        super(CombinedLoss, self).__init__()
        self.alpha = alpha
        self.beta = beta
        self.triplet = TripletLoss(margin=1.0)
        self.classification = nn.CrossEntropyLoss()
        
    def forward(self, anchor, positive, negative, classification_out, labels):
        triplet_loss = self.triplet(anchor, positive, negative)
        classification_loss = self.classification(classification_out, labels)
        total_loss = (self.alpha * triplet_loss) + (self.beta * classification_loss)
        
        return total_loss

In [14]:
class Net(nn.Module):
    def __init__(self, num_channels, num_classes):
        super(Net, self).__init__()
        
        self.conv1 = nn.Conv2d(num_channels, 96, kernel_size=7, stride=4)
        self.conv11 = nn.Conv2d(96, 256, kernel_size=1)
        self.conv2 = nn.Conv2d(96, 256, kernel_size=5, padding=2)
        self.conv22 = nn.Conv2d(256, 384, kernel_size=1)
        self.conv3 = nn.Conv2d(256, 384, kernel_size=3, padding=1)
        self.pool = nn.MaxPool2d(kernel_size=3, stride=2)
        self.norm = nn.LocalResponseNorm(5, alpha=0.0001, beta=0.75)
        
        self.fc6 = nn.Linear(in_features=13824, out_features=512)
        self.fc7 = nn.Linear(in_features=512, out_features=512)
        self.fc8 = nn.Linear(in_features=512, out_features=num_classes)
        
        self.relu = nn.ReLU()
        self.dropout = nn.Dropout(p=0.5)
        
    def forward(self, x, alpha: float):
        x = self.conv1(x)
        x = self.relu(x)
        x = self.pool(x)
        x = self.norm(x)
        
        sl1 = self.conv11(x)
        sl1 = self.relu(sl1)
        
        x = self.conv2(x)
        # make alpha trainable
        x = self.relu((1-alpha)*x + (alpha*sl1))
        # x = self.relu(x + sl1)
        x = self.pool(x)
        x = self.norm(x)
        
        sl2 = self.conv22(x)
        sl2 = self.relu(sl2)
        
        x = self.conv3(x)
        x = self.relu((1-alpha)*x + (alpha*sl2))
        # x = self.relu(x + sl2)
        x = self.pool(x)
        
        x = torch.flatten(x, 1)
        x = self.fc6(x)
        x = self.relu(x)
        x = self.dropout(x)
        
        x = self.fc7(x)
        x_triplet = x
        x = self.relu(x)
        x = self.dropout(x)
        
        x_classification = self.fc8(x)
        
        return x_triplet, x_classification

In [15]:
def init_weights(m):
    if isinstance(m, nn.Conv2d):
        torch.nn.init.kaiming_normal_(m.weight)

In [16]:
best_loss = 1.0
def objective(trial):
    model = Net(3, 2)
    model.apply(init_weights)
    model = torch.jit.script(model).to(device)
    
    lr = trial.suggest_float("learning_rate", 1e-5, 1e-2, log=True)
    mom = trial.suggest_float('momentum', 0.7, 0.99)
    wd = trial.suggest_float('weight_decay', 1e-5, 1e-1, log=True)
    optimizer = opt.SGD(model.parameters(), lr=lr, momentum=mom, weight_decay=wd)
    criterion = torch.jit.script(CombinedLoss())
    
    batch_size = trial.suggest_int('batch_size', 10, 100)
#     test_loader = DataLoader(test_data, batch_size=batch_size)
    
    num_epochs = 100
    alpha = trial.suggest_float('alpha', 0.1, 0.9, step=0.1)
    print("Learning rate: "+ str(lr))
    print("Momentum: "+ str(mom))
    print("Weight decay: "+ str(wd))
    print("Alpha: "+ str(alpha))
    print("Batch size: "+ str(batch_size))
    print("Number of epochs: "+ str(num_epochs))
    
    for epoch in tqdm(range(num_epochs), desc="Epochs"):
        train_loader = DataLoader(train_data, batch_size=batch_size, shuffle=True, num_workers=4)
        val_loader = DataLoader(val_data, batch_size=batch_size, shuffle=False, num_workers=4)
        
        # train(model, optim, train_loader)
        # training loop
        running_loss = []
        running_accu = []
        
        model.train()
        for step, (anchor_img, positive_img, negative_img, anchor_label) in enumerate(tqdm(train_loader, desc="Training", leave=False)):
            anchor_img, positive_img, negative_img, anchor_label = anchor_img.to(device), positive_img.to(device), negative_img.to(device), anchor_label.to(device)

            optimizer.zero_grad()
            anchor_out, anchor_pred = model(anchor_img, alpha)
            positive_out, _ = model(positive_img, alpha)
            negative_out, _ = model(negative_img, alpha)
            
            pred = torch.argmax(anchor_pred, 1)
            accuracy = torch.eq(pred, anchor_label).sum() / len(anchor_img)

#             print(anchor_out.shape, positive_out.shape, negative_out.shape, anchor_pred.shape, anchor_label.shape)
            loss = criterion(anchor_out, positive_out, negative_out, anchor_pred, anchor_label)
            loss.backward()
            optimizer.step()

            running_accu.append(accuracy.cpu().detach().numpy())
            running_loss.append(loss.cpu().detach().numpy())
        print("Epoch: {}/{} - Loss: {:.4f} - Accuracy: {:.4f}".format(epoch+1, num_epochs, np.mean(running_loss), np.mean(running_accu)))
        
        # accuracy = test(model, val_loader)
        # validation loop
        val_loss = []
        val_accu = []

        model.eval()
        with torch.no_grad():
            for img, pos_img, neg_img, label in tqdm(val_loader):
                img, pos_img, neg_img, label = img.to(device), pos_img.to(device), neg_img.to(device), label.to(device)
                
                anchor, output = model(img, alpha)
                pos, _ = model(pos_img, alpha)
                neg, _ = model(neg_img, alpha)
                
                pred = torch.argmax(output, 1)
                accuracy = torch.eq(pred, label).sum() / len(img)
                
                loss = criterion(anchor, pos, neg, output, label)
                
                val_accu.append(accuracy.cpu().detach().numpy())
                val_loss.append(loss.cpu().detach().numpy())
        val_accu = np.mean(val_accu)
        val_loss = np.mean(val_loss)
        print("Val Loss: {:.4f} - Val Accuracy: {:.4f}".format(val_loss, val_accu))
        
        trial.report(val_loss, epoch)
        if (epoch >= 50 and val_loss > 0.8) or trial.should_prune():
            raise optuna.exceptions.TrialPruned()
    
    global best_loss
    if val_loss < best_loss:
        best_loss = val_loss
        print("Saving best model...")
        torch.save(model.state_dict(), "../contrastive_skipmodel_alpha.pth")
            
    return val_loss

In [19]:
study = optuna.create_study(direction='minimize',
                            study_name='contrastive-skip-libfaceid-study',
                            storage='sqlite:///study.db',
                            load_if_exists=True)
study.optimize(objective, n_trials=10)

pruned_trials = study.get_trials(deepcopy=False, states=[TrialState.PRUNED])
complete_trials = study.get_trials(deepcopy=False, states=[TrialState.COMPLETE])

# Display the study statistics
print("\nStudy statistics: ")
print("  Number of finished trials: ", len(study.trials))
print("  Number of pruned trials: ", len(pruned_trials))
print("  Number of complete trials: ", len(complete_trials))

[I 2023-10-31 21:16:16,887] Using an existing study with name 'contrastive-skip-libfaceid-study' instead of creating a new one.


Learning rate: 1.9677052889502102e-05
Momentum: 0.7082202940071103
Weight decay: 1.3499632833993288e-05
Alpha: 0.30000000000000004
Batch size: 17
Number of epochs: 100


Epochs:   0%|          | 0/100 [00:00<?, ?it/s]

Training:   0%|          | 0/822 [00:00<?, ?it/s]

Epoch: 1/100 - Loss: 1.7041 - Accuracy: 0.5342


...

Training:   0%|          | 0/1398 [00:00<?, ?it/s]

Epoch: 64/100 - Loss: 0.5145 - Accuracy: 0.9292


  0%|          | 0/175 [00:00<?, ?it/s]

[I 2023-11-01 07:00:27,871] Trial 20 pruned. 


Val Loss: 0.8045 - Val Accuracy: 0.8749

Study statistics: 
  Number of finished trials:  21
  Number of pruned trials:  16
  Number of complete trials:  4


In [20]:
print("Best trial:")
trial = study.best_trial

print("Value: ", trial.value)

print("Params: ")
for key, value in trial.params.items():
    print("    {}: {}".format(key, value))

Best trial:
Value:  0.531609833240509
Params: 
    alpha: 0.1
    batch_size: 34
    learning_rate: 0.0010682255285837852
    momentum: 0.9149446504619274
    weight_decay: 0.017889481940035267


In [ ]:
# best alpha

Best trial:
Value:  0.531609833240509
Params: 
    alpha: 0.1
    batch_size: 34
    learning_rate: 0.0010682255285837852
    momentum: 0.9149446504619274
    weight_decay: 0.017889481940035267

In [18]:
# Alpha:  0.1

model = Net(3, 2)
model.apply(init_weights)
model = torch.jit.script(model).to(device)

batch_size = 26
num_epochs = 100

optimizer = opt.SGD(model.parameters(), lr=0.0032550730936762133, momentum=0.7767080035113286, weight_decay=0.011122487219970448)
criterion = torch.jit.script(CombinedLoss())

In [15]:
# Alpha:  0.2

model = Net(3, 2)
model.apply(init_weights)
model = torch.jit.script(model).to(device)

batch_size = 31
num_epochs = 100

optimizer = opt.SGD(model.parameters(), lr=0.0012467621126965754, momentum=0.9471065602440727, weight_decay=2.3711485262019323e-05)
criterion = torch.jit.script(CombinedLoss())

In [17]:
# Alpha:  0.3

model = Net(3, 2)
model.apply(init_weights)
model = torch.jit.script(model).to(device)

batch_size = 72
num_epochs = 100

optimizer = opt.SGD(model.parameters(), lr=0.007850045020228827, momentum=0.8227162376053277, weight_decay=0.0015734353330832543)
criterion = torch.jit.script(CombinedLoss())

In [15]:
# Alpha:  0.4

model = Net(3, 2)
model.apply(init_weights)
model = torch.jit.script(model).to(device)

batch_size = 44
num_epochs = 100

optimizer = opt.SGD(model.parameters(), lr=0.002573524369717596, momentum=0.7840526356490638, weight_decay=0.00048438154618545747)
criterion = torch.jit.script(CombinedLoss())

In [17]:
# Alpha:  0.5

model = Net(3, 2)
model.apply(init_weights)
model = torch.jit.script(model).to(device)

batch_size = 49
num_epochs = 100

optimizer = opt.SGD(model.parameters(), lr=0.0035132821482908817, momentum=0.920278581030804, weight_decay=6.599178316625366e-05)
criterion = torch.jit.script(CombinedLoss())

In [19]:
# Alpha:  0.6

model = Net(3, 2)
model.apply(init_weights)
model = torch.jit.script(model).to(device)

batch_size = 99
num_epochs = 100

optimizer = opt.SGD(model.parameters(), lr=0.008661653252197206, momentum=0.9336658754057047, weight_decay=0.00026102239516199836)
criterion = torch.jit.script(CombinedLoss())

In [21]:
# Alpha:  0.7

model = Net(3, 2)
model.apply(init_weights)
model = torch.jit.script(model).to(device)

batch_size = 70
num_epochs = 100

optimizer = opt.SGD(model.parameters(), lr=0.009366966400680977, momentum=0.7854191784490449, weight_decay=0.008407597494800292)
criterion = torch.jit.script(CombinedLoss())

In [23]:
# Alpha:  0.8

model = Net(3, 2)
model.apply(init_weights)
model = torch.jit.script(model).to(device)

batch_size = 42
num_epochs = 100

optimizer = opt.SGD(model.parameters(), lr=0.00984050059517122, momentum=0.7042475761046358, weight_decay=1.146204172707169e-05)
criterion = torch.jit.script(CombinedLoss())

In [25]:
# Alpha:  0.9

model = Net(3, 2)
model.apply(init_weights)
model = torch.jit.script(model).to(device)

batch_size = 61
num_epochs = 100

optimizer = opt.SGD(model.parameters(), lr=0.008032216546990966, momentum=0.9391670546476786, weight_decay=0.0003426507608107892)
criterion = torch.jit.script(CombinedLoss())

In [32]:
model = Net(3, 2)
model.apply(init_weights)
model = torch.jit.script(model).to(device)

batch_size = 30
num_epochs = 100

optimizer = opt.SGD(model.parameters(), lr=0.0004197845474223715, momentum=0.9430017883512166, weight_decay=0.013213204541489589)
criterion = torch.jit.script(CombinedLoss())

In [22]:
model = Net(3, 2)
model.apply(init_weights)
model = torch.jit.script(model).to(device)

batch_size = 33
num_epochs = 100

optimizer = opt.SGD(model.parameters(), lr=0.0017588542430838937, momentum=0.8796119877990213, weight_decay=0.008262294156468668)
criterion = torch.jit.script(CombinedLoss())

In [41]:
model = Net(3, 2)
model.apply(init_weights)
model = torch.jit.script(model).to(device)

batch_size = 65
num_epochs = 100

optimizer = opt.SGD(model.parameters(), lr=0.0024335191386082905, momentum=0.9443267966453481, weight_decay=0.004113118937673056)
criterion = torch.jit.script(CombinedLoss())

In [45]:
model = Net(3, 2)
model.apply(init_weights)
model = torch.jit.script(model).to(device)

batch_size = 21
num_epochs = 100

optimizer = opt.SGD(model.parameters(), lr=0.000504520824358099, momentum=0.9385824711132911, weight_decay=0.0001216664220206799)
criterion = torch.jit.script(CombinedLoss())

In [38]:
model = Net(3, 2)
model.apply(init_weights)
model = torch.jit.script(model).to(device)

batch_size = 72
num_epochs = 100

optimizer = opt.AdamW(model.parameters(), lr=0.0005)
criterion = torch.jit.script(CombinedLoss())

In [42]:
best_accu = 0.0
alpha = 0.3

for epoch in tqdm(range(num_epochs), desc="Epochs"):
    train_loader = DataLoader(train_data, batch_size=batch_size, shuffle=True, num_workers=4)
    val_loader = DataLoader(val_data, batch_size=batch_size, shuffle=False, num_workers=4)
    
    running_loss = []
    running_accu = []
    
    model.train()
    for step, (anchor_img, positive_img, negative_img, anchor_label) in enumerate(tqdm(train_loader, desc="Training", leave=False)):
        anchor_img, positive_img, negative_img, anchor_label = anchor_img.to(device), positive_img.to(device), negative_img.to(device), anchor_label.to(device)
        
        optimizer.zero_grad()
        anchor_out, anchor_pred = model(anchor_img, alpha)
        positive_out, _ = model(positive_img, alpha)
        negative_out, _ = model(negative_img, alpha)
        
        pred = torch.argmax(anchor_pred, 1)
        accuracy = torch.eq(pred, anchor_label).sum() / len(anchor_img)
        
        loss = criterion(anchor_out, positive_out, negative_out, anchor_pred, anchor_label)
        loss.backward()
        optimizer.step()
        
        running_accu.append(accuracy.cpu().detach().numpy())
        running_loss.append(loss.cpu().detach().numpy())
    print("Epoch: {}/{} - Loss: {:.4f} - Accuracy: {:.4f}".format(epoch+1, num_epochs, np.mean(running_loss), np.mean(running_accu)))
    
    val_loss = []
    val_accu = []

    model.eval()
    with torch.no_grad():
        for img, pos_img, neg_img, label in tqdm(val_loader):
            img, pos_img, neg_img, label = img.to(device), pos_img.to(device), neg_img.to(device), label.to(device)

            anchor, output = model(img, alpha)
            pos, _ = model(pos_img, alpha)
            neg, _ = model(neg_img, alpha)

            pred = torch.argmax(output, 1)
            accuracy = torch.eq(pred, label).sum() / len(img)

            loss = criterion(anchor, pos, neg, output, label)

            val_accu.append(accuracy.cpu().detach().numpy())
            val_loss.append(loss.cpu().detach().numpy())
    val_accu = np.mean(val_accu)
    val_loss = np.mean(val_loss)
    print("Val Loss: {:.4f} - Val Accuracy: {:.4f}".format(val_loss, val_accu))
    
    if val_accu > best_accu:
        best_accu = val_accu
        print("Saving best model...")
        torch.save(model.state_dict(), "../contrastive_skipmodel_3.pth")

Epochs:   0%|          | 0/100 [00:00<?, ?it/s]

Training:   0%|          | 0/215 [00:00<?, ?it/s]

Epoch: 1/100 - Loss: 1.7015 - Accuracy: 0.5371


  0%|          | 0/27 [00:00<?, ?it/s]

Val Loss: 1.6839 - Val Accuracy: 0.5354
Saving best model...


...

Val Loss: 0.5392 - Val Accuracy: 0.9267


Training:   0%|          | 0/215 [00:00<?, ?it/s]

Epoch: 100/100 - Loss: 0.0596 - Accuracy: 0.9951


  0%|          | 0/27 [00:00<?, ?it/s]

Val Loss: 0.5793 - Val Accuracy: 0.9312


In [ ]:
model = Net(3, 2).to(device)
checkpoint = torch.load("../contrastive_skipmodel_3.pth")
model.load_state_dict(checkpoint)

alpha = 0.3
test_loader = DataLoader(test_data, batch_size=65, shuffle=False, num_workers=4)
test_accu = []

model.eval()
with torch.no_grad():
    for img, label in tqdm(test_loader):
        img, label = img.to(device), label.to(device)

        _, output = model(img, alpha)
        pred = torch.argmax(output, 1)
        accuracy = torch.eq(pred, label).sum() / len(img)

        test_accu.append(accuracy.cpu().detach().numpy())
test_accu = np.mean(test_accu)
print("Test Accuracy: {:.4f}".format(test_accu))

In [ ]:
0.1 -- 92.10
0.2 -- 93.07
0.3 -- 93.93
0.4 -- 93.21
0.5 -- 93.05
0.6 -- 93.50
0.7 -- 92.72
0.8 -- 91.73
0.9 -- 89.55